In [1]:
import IJulia

# The julia kernel has built in support for Revise.jl, so this is the 
# recommended approach for long-running sessions:
# https://github.com/JuliaLang/IJulia.jl/blob/9b10fa9b879574bbf720f5285029e07758e50a5e/src/kernel.jl#L46-L51

# Users should enable revise within .julia/config/startup_ijulia.jl:
# https://timholy.github.io/Revise.jl/stable/config/#Using-Revise-automatically-within-Jupyter/IJulia-1

# clear console history
IJulia.clear_history()

fig_width = 7
fig_height = 5
fig_format = :retina
fig_dpi = 96

# no retina format type, use svg for high quality type/marks
if fig_format == :retina
  fig_format = :svg
elseif fig_format == :pdf
  fig_dpi = 96
  # Enable PDF support for IJulia
  IJulia.register_mime(MIME("application/pdf"))
end

# convert inches to pixels
fig_width = fig_width * fig_dpi
fig_height = fig_height * fig_dpi

# Intialize Plots w/ default fig width/height
try
  import Plots

  # Plots.jl doesn't support PDF output for versions < 1.28.1
  # so use png (if the DPI remains the default of 300 then set to 96)
  if (Plots._current_plots_version < v"1.28.1") & (fig_format == :pdf)
    Plots.gr(size=(fig_width, fig_height), fmt = :png, dpi = fig_dpi)
  else
    Plots.gr(size=(fig_width, fig_height), fmt = fig_format, dpi = fig_dpi)
  end
catch e
  # @warn "Plots init" exception=(e, catch_backtrace())
end

# Initialize CairoMakie with default fig width/height
try
  import CairoMakie

  # CairoMakie's display() in PDF format opens an interactive window
  # instead of saving to the ipynb file, so we don't do that.
  # https://github.com/quarto-dev/quarto-cli/issues/7548
  if fig_format == :pdf
    CairoMakie.activate!(type = "png")
  else
    CairoMakie.activate!(type = string(fig_format))
  end
  CairoMakie.update_theme!(resolution=(fig_width, fig_height))
catch e
    # @warn "CairoMakie init" exception=(e, catch_backtrace())
end
  
# Set run_path if specified
try
  run_path = raw"/Users/hirofumi48/162348.github.io/posts/2025/Comp"
  if !isempty(run_path)
    cd(run_path)
  end
catch e
  @warn "Run path init:" exception=(e, catch_backtrace())
end


# emulate old Pkg.installed beahvior, see
# https://discourse.julialang.org/t/how-to-use-pkg-dependencies-instead-of-pkg-installed/36416/9
import Pkg
function isinstalled(pkg::String)
  any(x -> x.name == pkg && x.is_direct_dep, values(Pkg.dependencies()))
end

# ojs_define
if isinstalled("JSON") && isinstalled("DataFrames")
  import JSON, DataFrames
  global function ojs_define(; kwargs...)
    convert(x) = x
    convert(x::DataFrames.AbstractDataFrame) = Tables.rows(x)
    content = Dict("contents" => [Dict("name" => k, "value" => convert(v)) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
elseif isinstalled("JSON")
  import JSON
  global function ojs_define(; kwargs...)
    content = Dict("contents" => [Dict("name" => k, "value" => v) for (k, v) in kwargs])
    tag = "<script type='ojs-define'>$(JSON.json(content))</script>"
    IJulia.display(MIME("text/html"), tag)
  end
else
  global function ojs_define(; kwargs...)
    @warn "JSON package not available. Please install the JSON.jl package to use ojs_define."
  end
end


# don't return kernel dependencies (b/c Revise should take care of dependencies)
nothing


In [2]:
#| output: false
n, p, pₑ = 200, 50, 10

using Random, StatsFuns, Distributions
β_true = vcat(randn(pₑ), zeros(p - pₑ))
X = randn(n, p)

η_true = X * β_true
π_true = logistic.(η_true)

y = rand.(Bernoulli.(π_true))
y = collect(Float64, y)

200-element Vector{Float64}:
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 1.0
 1.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 1.0
 1.0
 0.0
 0.0
 0.0
 0.0

In [3]:
#| output: false
using PolyaGammaHybridSamplers, LinearAlgebra, MCMCChains, Dates, MCMCDiagnosticTools

function pg_logistic_gibbs(
  X::Matrix{Float64},
  y::Vector{Float64};
  n_samples::Int = 5000,
  burnin::Int = 1000,
  σ_prior::Float64 = 10.0,
)
  n, p = size(X)

  # 事前: β ~ N(0, σ_prior^2 I)
  V0_inv = (1 / σ_prior^2) * LinearAlgebra.I  # precision of prior

  # 初期値
  β = zeros(p)
  κ = y .- 0.5  # κ_i = y_i - 1/2

  # サンプル保存用
  n_iter = n_samples + burnin
  β_samples = Matrix{Float64}(undef, n_samples, p)

  t_start = time()
  for it in 1:n_iter
    # 1. PG 補助変数 ω_i | β のサンプル
    η = X * β
    ω = similar(η)
    for i in 1:n
      pg = PolyaGammaHybridSampler(1.0, η[i])
      ω[i] = rand(pg)
    end

    # 2. β | ω, y のサンプル (多変量ガウス)
    Ω = Diagonal(ω)
    precision = X' * Ω * X + V0_inv          # posterior precision
    cov = inv(Matrix(precision))             # posterior covariance
    m = cov * (X' * κ)                       # posterior mean (μ0=0 のため)

    # β ~ N(m, cov)
    β = rand(MvNormal(m, Symmetric(cov)))

    # burn-in 後に保存
    if it > burnin
      β_samples[it - burnin, :] .= β
    end
  end
  t_stop = time()
  runtime_sec = t_stop - t_start

  names = Symbol.("β[$i]" for i in 1:p)
  values = reshape(β_samples, n_samples, p, 1)
  chain = Chains(values, names)
  chain = setinfo(chain, (
    start_time = [t_start],  # 1本チェインなら長さ1のベクトルでOK
    stop_time  = [t_stop],
  ))

  return chain, runtime_sec
end

pg_logistic_gibbs (generic function with 1 method)

In [4]:
σ_prior = 10.0
chain_pg, t_pg = pg_logistic_gibbs(X, y;
    n_samples = 3000,
    burnin = 3000,
    σ_prior = σ_prior,
)
summarize(chain_pg)



  parameters      mean       std      mcse   ess_bulk   ess_tail      rhat   e ⋯
      Symbol   Float64   Float64   Float64    Float64    Float64   Float64     ⋯

        β[1]    5.0938    1.0469    0.1640    41.4443   170.0661    1.0050     ⋯
        β[2]    0.6911    0.6890    0.0412   279.7676   604.9084    1.0023     ⋯
        β[3]   -0.7582    0.6453    0.0313   432.3889   688.4741    1.0019     ⋯
        β[4]    9.6110    1.6016    0.2042    60.8553   248.3164    1.0094     ⋯
        β[5]    3.2738    0.7971    0.0690   130.5799   429.7454    1.0046     ⋯
        β[6]   -9.2844    1.6218    0.2528    41.1782   193.8003    1.0149     ⋯
        β[7]    6.3970    1.1865    0.1728    47.5656   245.1486    1.0057     ⋯
        β[8]   -1.0989    0.6198    0.0364   298.6133   526.0545    1.0017     ⋯
        β[9]    9.5024    1.5085    0.2210    47.1074   201.7003    1.0131     ⋯
       β[10]    0.9900    0.8524    0.0484   310.7054   715.2504    1.0000     ⋯
       β[11]   -0.2928   

In [5]:
#| output: false
using Turing, LinearAlgebra

@model function logreg_turing(x, y, σ_prior)
    n, p = size(x)
    
    # 事前分布
    β ~ MvNormal(zeros(p), (σ_prior^2) * I)
    
    # ベクトル化した尤度（高速化）
    η = x * β
    y ~ arraydist(Bernoulli.(logistic.(η)))
end

model = logreg_turing(X, y, σ_prior)

DynamicPPL.Model{typeof(logreg_turing), (:x, :y, :σ_prior), (), (), Tuple{Matrix{Float64}, Vector{Float64}, Float64}, Tuple{}, DynamicPPL.DefaultContext}(Main.logreg_turing, (x = [2.2051425958466973 1.4728971151565833 … 0.8944236195675717 -0.8807630871138868; -0.36991146408653414 -0.748132884881536 … 0.577482609113408 -0.6055709202769083; … ; 0.18884591116435417 -0.49184285126350497 … 0.3346757359465313 -0.503343053134923; -0.41710443450645096 -0.4409145445271303 … 0.7195094047203687 -0.15769889533619638], y = [1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0  …  1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0], σ_prior = 10.0), NamedTuple(), DynamicPPL.DefaultContext())

In [6]:
#| output: false
n_samples = 3000
n_adapt   = 3000

chain_hmc = sample(
    model,
    NUTS(n_adapt, 0.6),
    n_samples,
)

┌ Info: Found initial step size
└   ϵ = 0.8


Sampling:   2%|▋                                        |  ETA: 0:00:23

Sampling:   2%|▉                                        |  ETA: 0:00:26

Sampling:   2%|█                                        |  ETA: 0:00:29

Sampling:   3%|█▎                                       |  ETA: 0:00:30

...

Sampling:  98%|████████████████████████████████████████▍|  ETA: 0:00:00

Sampling:  99%|████████████████████████████████████████▋|  ETA: 0:00:00

Sampling:  99%|████████████████████████████████████████▊|  ETA: 0:00:00

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:31

Chains MCMC chain (3000×64×1 Array{Float64, 3}):

Iterations        = 3001:1:6000
Number of chains  = 1
Samples per chain = 3000
Wall duration     = 32.81 seconds
Compute duration  = 32.81 seconds
parameters        = β[1], β[2], β[3], β[4], β[5], β[6], β[7], β[8], β[9], β[10], β[11], β[12], β[13], β[14], β[15], β[16], β[17], β[18], β[19], β[20], β[21], β[22], β[23], β[24], β[25], β[26], β[27], β[28], β[29], β[30], β[31], β[32], β[33], β[34], β[35], β[36], β[37], β[38], β[39], β[40], β[41], β[42], β[43], β[44], β[45], β[46], β[47], β[48], β[49], β[50]
internals         = n_steps, is_accept, acceptance_rate, log_density, hamiltonian_energy, hamiltonian_energy_error, max_hamiltonian_energy_error, tree_depth, numerical_error, step_size, nom_step_size, lp, logprior, loglikelihood

Use `describe(chains)` for summary statistics and quantiles.


In [7]:
summarize(chain_hmc)



  parameters      mean       std      mcse    ess_bulk    ess_tail      rhat   ⋯
      Symbol   Float64   Float64   Float64     Float64     Float64   Float64   ⋯

        β[1]    4.8971    1.1114    0.0365    930.7941   1588.1634    0.9998   ⋯
        β[2]    0.6469    0.6489    0.0131   2445.7267   2123.6187    1.0008   ⋯
        β[3]   -0.6992    0.6723    0.0139   2355.6245   2262.1939    1.0013   ⋯
        β[4]    9.2684    1.8183    0.0640    822.0774   1250.5017    0.9997   ⋯
        β[5]    3.1613    0.8446    0.0259   1088.2547   1823.5098    1.0003   ⋯
        β[6]   -8.9051    1.7579    0.0635    780.6818   1268.6961    1.0027   ⋯
        β[7]    6.1628    1.3241    0.0459    846.6077   1284.8146    1.0001   ⋯
        β[8]   -1.0952    0.6145    0.0136   2070.1264   2358.9584    0.9997   ⋯
        β[9]    9.1594    1.6972    0.0622    762.3070   1316.5157    1.0000   ⋯
       β[10]    0.9604    0.8599    0.0169   2588.7571   2614.2082    1.0002   ⋯
       β[11]   -0.3022   

In [8]:
using Statistics

# 真の β との誤差
mean_hmc = vec(mean(Array(chain_hmc), dims=1))  # ここは実際のパラメータ名に合わせて調整
mean_pg = vec(mean(Array(chain_pg), dims=1))

println("‖β̂_HMC - β_true‖₂ = ", norm(mean_hmc .- β_true))
println("‖β̂_PG  - β_true‖₂ = ", norm(mean_pg  .- β_true))

# ランタイムや ESS の比較も：
ess_hmc = ess_rhat(chain_hmc)
ess_pg  = ess_rhat(chain_pg)
println("ESS (HMC) = ", mean(ess_hmc[:,:ess]))
println("ESS (PG) = ", mean(ess_pg[:,:ess]))
println("ESS/s (HMC) = ", mean(ess_hmc[:,:ess_per_sec]))
println("ESS/s (PG) = ", mean(ess_pg[:,:ess_per_sec]))

‖β̂_HMC - β_true‖₂ = 17.631999619963004
‖β̂_PG  - β_true‖₂ = 18.435891995080777
ESS (HMC) = 1853.1003569246561


ESS (PG) = 238.70064632682056
ESS/s (HMC) = 56.478021301534724
ESS/s (PG) = 146.71213664832237
